In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

global num
num=str(3)
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = np.loadtxt('distance_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['distance_matrix']=np.dot(data['distance_matrix'],np.diag([100000000000000] * len(data['distance_matrix'])))
    demands=[]
    for i in np.loadtxt('demand_'+str(num)+'.txt', dtype=np.float32, delimiter=','):
        demands.append(int(i[1]))
    data['demands'] = demands
    
    w=np.loadtxt('spec_'+str(num)+'.txt', dtype=np.float32, delimiter=',')
    data['vehicle_capacities'] = []
    data['num_vehicles'] = int(w[1])
    for j in range(int(w[1])):
        data['vehicle_capacities'].append(int(w[2]))
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    
    f=open("solution_"+num+".txt","w")
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        l=[]
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            l.append(int('{0}'.format(node_index, route_load)))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        for i in range(len(l)):
            l[i]=l[i]+1
        l.append(1)
        kk=0
        for i in l:
            kk+=1
            if kk==len(l):
                f.write(str(i))
            else:
                f.write(str(i)+',')
        f.write('\n')
        print(l)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    f.close()


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = True
    search_parameters.use_full_propagation=True
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 72354123581387091
[1, 10, 16, 2, 3, 17, 25, 1]
Route for vehicle 0:
 0 Load(0) ->  9 Load(165) ->  15 Load(243) ->  1 Load(276) ->  2 Load(291) ->  16 Load(918) ->  24 Load(1612) ->  0 Load(1612)
Distance of the route: 2472380555607376m
Load of the route: 1612

[1, 43, 42, 31, 33, 32, 35, 41, 40, 37, 39, 38, 1]
Route for vehicle 1:
 0 Load(0) ->  42 Load(66) ->  41 Load(304) ->  30 Load(421) ->  32 Load(1721) ->  31 Load(1773) ->  34 Load(1801) ->  40 Load(1842) ->  39 Load(1930) ->  36 Load(1931) ->  38 Load(1950) ->  37 Load(2004) ->  0 Load(2004)
Distance of the route: 11346693158149717m
Load of the route: 2004

[1, 44, 45, 29, 34, 30, 28, 7, 6, 8, 36, 4, 5, 15, 14, 13, 12, 19, 18, 11, 1]
Route for vehicle 2:
 0 Load(0) ->  43 Load(44) ->  44 Load(86) ->  28 Load(192) ->  33 Load(249) ->  29 Load(301) ->  27 Load(482) ->  6 Load(487) ->  5 Load(502) ->  7 Load(579) ->  35 Load(663) ->  3 Load(673) ->  4 Load(713) ->  14 Load(1263) ->  13 Load(1281) ->  12 Load(1304) ->  1